In [ ]:
import json
import torch
import torch.nn as nn
from TransformerEncoder import TransformerEncoder
model = TransformerEncoder()
model_size = sum(p.numel() for p in model.parameters()) / (1024 * 1024)
model

In [ ]:
print("Model size: {:.3f} MB".format(model_size))

In [ ]:
data = "./data.json"
f = open(data)
data = json.load(f)
train_data = data["data"]["train"]
train_data

In [ ]:
label = data["data"]["label"]
label

In [ ]:
cv_data = data["data"]["valid"]
cv_data

In [ ]:
cv_label = data["data"]["v_label"]
cv_label

In [ ]:
lossfn = nn.CrossEntropyLoss()
pred_logmax = nn.LogSoftmax(dim=1)
lr = 1e-4
optermizer = torch.optim.Adam(model.parameters(), lr=lr)
data = torch.tensor(train_data)
data

In [ ]:

label = torch.tensor(label)
label

In [ ]:
cv_data = torch.tensor(cv_data)
cv_data

In [ ]:
cv_label = torch.tensor(cv_label)
cv_label

In [ ]:
loss_train = []
loss_cv = []
for epoch in range(2):
    for i in range(1000):
        model.train()
        pred = model(data)
        pred = pred.transpose(2, 1)
        loss = lossfn(pred, label)
        optermizer.zero_grad()
        loss.backward()
        optermizer.step()
        if i % 100 == 0:
            print("train_loss: {:.3f}".format(loss.detach().float().tolist()))
            loss_train.append(loss.item())
    
        model.eval()
        with torch.no_grad():
            cv_pred = model(cv_data)
            cv_pred = cv_pred.transpose(2, 1)
            cv_loss = lossfn(cv_pred, cv_label)
            if i%100==0:
                print("cv_loss: {:.3f}".format(cv_loss.detach().float().tolist()))
                loss_cv.append(cv_loss.item())

In [ ]:

from matplotlib.pylab import plt
from numpy import arange
import numpy as np
plt.plot(np.array(loss_train), label='Training Loss')
plt.plot(np.array(loss_cv), label='validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(arange(0, 20, 2))
plt.legend(loc='best')
plt.show()